In [1]:
import torch
import os

print("🧠 PyTorch 버전:", torch.__version__)
print("🚀 CUDA 사용 가능:", torch.cuda.is_available())
print("💻 GPU 이름:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "사용 불가")
print("📦 CUDA 버전 (PyTorch에서 사용 중인):", torch.version.cuda)
print("🔧 cuDNN 버전:", torch.backends.cudnn.version())
print("🔎 cuDNN 사용 가능:", torch.backends.cudnn.enabled)


🧠 PyTorch 버전: 2.7.0+cu118
🚀 CUDA 사용 가능: True
💻 GPU 이름: NVIDIA GeForce RTX 3090 Ti
📦 CUDA 버전 (PyTorch에서 사용 중인): 11.8
🔧 cuDNN 버전: 90100
🔎 cuDNN 사용 가능: True


In [ ]:
# ✅ 설치 필요시
# pip install transformers datasets

import torch
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    TrainerCallback,
)

# ✅ 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ CUDA 사용:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("🖥️ GPU:", torch.cuda.get_device_name(0))

# ✅ 모델 및 토크나이저 로드
model_name = "paust/pko-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer.pad_token = tokenizer.eos_token

# ✅ 데이터셋 로드
df = pd.read_csv(r"C:\Users\AI-LJH\Desktop\cap\smilestyle_dataset_filtered_cleaned.tsv", sep="\t")
df = df.dropna(thresh=3)

# ✅ 학습 샘플 생성 함수
def build_samples(df):
    samples = []
    for _, row in df.iterrows():
        row = row.dropna()
        for tgt_col in row.index:
            tgt = row[tgt_col]
            for src_col in row.index:
                if src_col == tgt_col:
                    continue
                src = row[src_col]
                prompt = f"Translate to {tgt_col} style: {src}"
                samples.append((prompt, tgt))
    return samples

samples = build_samples(df)

# ✅ 커스텀 Dataset
class StyleDataset(Dataset):
    def __init__(self, samples, tokenizer, max_length=128):
        self.samples = samples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        src, tgt = self.samples[idx]
        inputs = self.tokenizer(
            src, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt"
        )
        labels = self.tokenizer(
            tgt, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt"
        )["input_ids"]
        labels[labels == tokenizer.pad_token_id] = -100
        inputs["labels"] = labels
        return {k: v.squeeze() for k, v in inputs.items()}

# ✅ 학습/검증 분할
train_samples, eval_samples = train_test_split(samples, test_size=0.1, random_state=42)
train_dataset = StyleDataset(train_samples, tokenizer)
eval_dataset = StyleDataset(eval_samples, tokenizer)

# ✅ 100 step마다 실시간 로그 출력 콜백
class PrintLossCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        current = state.global_step
        total = state.max_steps
        if logs is not None:
            if "loss" in logs:
                print(f"🟠 Step {current:>5} / {total}: Train Loss = {logs['loss']:.4f}")
            if "eval_loss" in logs:
                print(f"🟢 Step {current:>5} / {total}: Eval  Loss = {logs['eval_loss']:.4f}")


# ✅ 학습 인자
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_style_fildata_finetuned",
    eval_strategy="steps",          
    eval_steps=1000,                      # <- 1000 스텝마다 평가
    save_strategy="steps",
    save_steps=1000,
    logging_steps=1000,
    learning_rate=5e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=20,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=True,
    report_to="none",
    disable_tqdm=False,
)


# ✅ Trainer 구성
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    callbacks=[PrintLossCallback()]
)

# ✅ 학습 시작
trainer.train()

# ✅ 모델 저장
trainer.save_model("./t5_style_finetuned/final_model")
tokenizer.save_pretrained("./t5_style_finetuned/final_model")


✅ CUDA 사용: True
🖥️ GPU: NVIDIA GeForce RTX 3090 Ti


C:\Users\AI-LJH\AppData\Local\Temp\ipykernel_22724\236329852.py:113: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [7]:
import torch
import re
from kss import split_sentences
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ✅ 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 스타일 목록 정의
styles = [
    'formal', 'informal', 'android', 'enfp', 'gentle',
    'halbae', 'king', 'sosim', 'translator'
]

style_kor_map = {
    'formal': '문어체', 'informal': '구어체', 'android': '안드로이드',
    'enfp': 'enfp', 'gentle': '신사', 'halbae': '할아버지',
    'king': '왕', 'sosim': '소심한', 'translator': '번역기'
}

# ✅ 모델 로딩
MODEL_PATH = r"C:\Users\AI-LJH\Desktop\캡스톤\t5_style_finetuned\final_model"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH).to(device)

# ✅ 후처리 필터 함수
def clean_generated(text):
    text = re.sub(r'(.)\1{3,}', r'\1\1', text)  # 4자 이상 반복 제거
    text = re.sub(r'[^\w\s가-힣.,?!~\'\"()\[\]]+', '', text)  # 특수문자 제거
    text = re.sub(r'([.,?!])\1+', r'\1', text)  # 구두점 반복 제거
    text = re.sub(r'(\b.+?\b)( \1)+', r'\1', text)  # 문장 반복 제거
    return text.strip()

# ✅ 스타일 변환 함수
def style_transfer(sentence, target_style, max_length=32, num_beams=5):
    prompt = f"{target_style} 말투로 변환: {sentence}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=max_length)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_beams=num_beams,
            repetition_penalty=4.0,
            no_repeat_ngram_size=5,
            length_penalty=1.2,
            early_stopping=True,
        )

    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return clean_generated(result)

# ✅ 전체 스타일 변환 수행 함수
def convert_all_styles(text):
    sentences = split_sentences(text)
    results = {}

    for style in styles:
        styled_sentences = [style_transfer(s, style) for s in sentences]
        results[style] = " ".join(styled_sentences)

    return results

# ✅ 테스트 문장
input_text = (
    "오늘은 정말 정신없는 하루였다. 회의가 계속 이어졌고, 중요한 보고서도 마무리해야 했다. "
    "지친 몸을 이끌고 집에 돌아오니 비로소 마음이 편안해졌다."
)

# ✅ 실행
outputs = convert_all_styles(input_text)

# ✅ 출력
for style in styles:
    print(f"🗂️ [{style}] {style_kor_map[style]}")
    print(outputs[style])
    print("-" * 80)


🗂️ [formal] 문어체
오늘 안색이 안 좋은 것 같아요. ᅲᅲᅲ 제발 오늘은 그렇게 말하지 말아주세요. 회의가 계속 되면서 중요한 자료들은 다 들고 나오려고 했어요. 참고로 오늘 아침 지친 몸을 이끌고 집에 돌아오니 마음이 평온해지는 것 같아요. 기분이 좋습니다. 다시
--------------------------------------------------------------------------------
🗂️ [informal] 구어체
오늘은 정말 하루가 힘드네. ᅲᅲᅲ 제발 오늘은 그렇게 말하지 말아줘. 난 오늘 회의가 계속 되면서 중요한 자료들도 마저 정리해야 했어. 안녕하세요. 그리고 중요한 지친 몸을 이끌고 집에 돌아오니 마음이 평온해지는 것 같아. 오랜만에 느끼는 마음이야.
--------------------------------------------------------------------------------
🗂️ [android] 안드로이드
대화방식. 잘못됨. 오늘. 일. 힘듬. 예상됨. ᅲᅲᅲ. 진심. 바쁨. 회의. 계속됨. 중요한. 자료. 처리. 필요했음. ᄏᄏᄏ. 마무리. 사진. 안드로이드. 어제. 일함. 그러나. 집. 복귀. 마음. 평온해짐. 느낌. 좋음.
--------------------------------------------------------------------------------
🗂️ [enfp] enfp
오늘 아침 날씨가 너~무 좋은 거 있지?! ᄒᄒ 고마웡!ᄒ 회의가 계속 될수록 중요한 자료들은 다 들구 나오려고 했엉 ᄒᄒ 중요한 힘든 맘을 뒤로하고 집에 온 뒤로 마음이 평온해진 거 가타!ᄒᄒ
--------------------------------------------------------------------------------
🗂️ [gentle] 신사
오늘은 조금 더 정신없는 하루네요. ᄒᄒ, 오늘은 정말 쉴 틈이 회의가 계속 되면서 중요한 자료들을 다 들고 나오려 했습니다. ᄒᄒ 감사한 